## Neural Collaborative Filtering
Reference:
* Paper: Xiangnan He, Lizi Liao, Hanwang Zhang, Liqiang Nie, Xia Hu and Tat-Seng Chua (2017) https://arxiv.org/abs/1708.05031
* Code with the paper: https://github.com/hexiangnan/neural_collaborative_filtering

Here I implemented their ideas using Keras with tensorflow backend on a smaller MovieLens 100K dataset.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
import keras
from keras.layers import Input, Embedding, Dropout, Flatten, concatenate, dot, BatchNormalization, Dense
from keras import Model

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

Using TensorFlow backend.


In [2]:
ratings = pd.read_csv('data/ratings_small.csv')
movies = pd.read_csv('data/movies_small.csv')

In [3]:
n_users = ratings.userId.unique().shape[0]
n_movies = ratings.movieId.unique().shape[0]
print('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_movies))

Number of users = 610 | Number of movies = 9724


In [4]:
# assign a unique number between (0, #users) to each user and do the same for movies
ratings.userId = ratings.userId.astype('category').cat.codes.values
ratings.movieId = ratings.movieId.astype('category').cat.codes.values

In [5]:
train, test = train_test_split(ratings, test_size=0.2)

In [6]:
train.head()

,userId,movieId,rating,timestamp
18202,114,1287,4.0,957647369
65102,415,906,4.0,1187497538
57229,379,2093,4.0,1493474361
24874,175,253,3.0,840108793
24058,166,97,3.5,1154721971


In [7]:
test.head()

,userId,movieId,rating,timestamp
23744,162,1208,4.0,894217570
92857,598,483,3.5,1498500822
15234,97,8676,5.0,1532457746
12840,81,2248,3.5,1084468057
66117,424,3633,4.5,1085490522


In [8]:
y_rating = test.rating

#### Construct the model

In [9]:
n_latent_factors_user = 8
n_latent_factors_movie = 10
n_latent_factors_mf = 3

In [10]:
movie_input = Input(shape=[1],name='Item')
movie_embedding_mlp = Embedding(n_movies + 1, n_latent_factors_movie, name='Movie-Embedding-MLP')(movie_input)
movie_vec_mlp = Flatten(name='FlattenMovies-MLP')(movie_embedding_mlp)
movie_vec_mlp = Dropout(0.2)(movie_vec_mlp)

movie_embedding_mf = Embedding(n_movies + 1, n_latent_factors_mf, name='Movie-Embedding-MF')(movie_input)
movie_vec_mf = Flatten(name='FlattenMovies-MF')(movie_embedding_mf)
movie_vec_mf = Dropout(0.2)(movie_vec_mf)


user_input = Input(shape=[1],name='User')
user_vec_mlp = Flatten(name='FlattenUsers-MLP')(Embedding(n_users + 1, n_latent_factors_user,name='User-Embedding-MLP')(user_input))
user_vec_mlp = Dropout(0.2)(user_vec_mlp)

user_vec_mf = Flatten(name='FlattenUsers-MF')(Embedding(n_users + 1, n_latent_factors_mf,name='User-Embedding-MF')(user_input))
user_vec_mf = Dropout(0.2)(user_vec_mf)


concat = concatenate([movie_vec_mlp, user_vec_mlp], name='Concat')
concat_dropout = Dropout(0.2)(concat)
dense = Dense(200,name='FullyConnected')(concat_dropout)
dense_batch = BatchNormalization(name='Batch')(dense)
dropout_1 = Dropout(0.2,name='Dropout-1')(dense_batch)
dense_2 = Dense(100,name='FullyConnected-1')(dropout_1)
dense_batch_2 = BatchNormalization(name='Batch-2')(dense_2)
dropout_2 = Dropout(0.2,name='Dropout-2')(dense_batch_2)
dense_3 = Dense(50,name='FullyConnected-2')(dropout_2)
dense_4 = Dense(20,name='FullyConnected-3', activation='relu')(dense_3)

pred_mf = dot([movie_vec_mf, user_vec_mf], axes = -1, name='Dot')


pred_mlp = Dense(1, activation='relu',name='Activation')(dense_4)

combine_mlp_mf = concatenate([pred_mf, pred_mlp], name='Concat-MF-MLP')
result_combine = Dense(100,name='Combine-MF-MLP')(combine_mlp_mf)
deep_combine = Dense(100,name='FullyConnected-4')(result_combine)


result = Dense(1, activation='relu', name='Prediction')(deep_combine)


model = Model([user_input, movie_input], result)
# opt = keras.optimizers.Adam(lr =0.01)
model.compile(optimizer='adam',loss= 'mean_squared_error')

In [11]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Item (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
User (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
Movie-Embedding-MLP (Embedding) (None, 1, 10)        97250       Item[0][0]                       
__________________________________________________________________________________________________
User-Embedding-MLP (Embedding)  (None, 1, 8)         4888        User[0][0]                       
__________________________________________________________________________________________________
FlattenMov

In [12]:
history = model.fit([train.userId, train.movieId], train.rating, epochs=20, verbose=1, validation_split=0.1)

Train on 72601 samples, validate on 8067 samples
Epoch 1/20
72601/72601 [==============================] - 44s 613us/step - loss: 1.1320 - val_loss: 0.9555
Epoch 2/20
72601/72601 [==============================] - 38s 522us/step - loss: 0.8796 - val_loss: 0.8161
Epoch 3/20
72601/72601 [==============================] - 38s 525us/step - loss: 0.7742 - val_loss: 0.8068
Epoch 4/20
72601/72601 [==============================] - 37s 508us/step - loss: 0.7196 - val_loss: 0.7938
Epoch 5/20
72601/72601 [==============================] - 37s 511us/step - loss: 0.6846 - val_loss: 0.7763
Epoch 6/20
72601/72601 [==============================] - 37s 509us/step - loss: 0.6637 - val_loss: 0.7706
Epoch 7/20
72601/72601 [==============================] - 39s 530us/step - loss: 0.6486 - val_loss: 0.7733
Epoch 8/20
72601/72601 [==============================] - 39s 533us/step - loss: 0.6299 - val_loss: 0.7710
Epoch 9/20
72601/72601 [==============================] - 38s 529us/step - loss: 0.6235 - val_l

In [13]:
rmse = sqrt(mean_squared_error(y_rating, model.predict([test.userId, test.movieId])))
print(rmse)

0.8806567962801158
